In [ ]:

#Used for storing and playing with data using DataFrame.
import pandas as pd
#Used for working with arrays in python.
import numpy as np
#Used for plotting graphs from data.
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
!pip install geopandas
import plotly.express as px
import geopandas as gpd
#Used for creating ML models.
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.tree import plot_tree
#used for train test splitting
from sklearn.model_selection import train_test_split
#Distrubation test
import scipy
from scipy.stats import skew, kurtosis, shapiro, jarque_bera
#Normalization 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
# Read csv data
data = pd.read_csv('RIA02.20230502T160508.csv')
#For Europe map and Anova analysis
world_data=pd.read_excel("prc_colc_rents_page_spreadsheet.xlsx")
#alpha_code=pd.read_excel("Alpha_codes.xlsx")
#Anova Test
from scipy.stats import f_oneway
#Tukey HDS Test
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd




### Number of Bedrooms
One bed :1
Two bed : 2
Three bed : 3
Four plus bed: 4 

### Property Type
Other flats: 1
Apartment: 2
Terrace house : 3
Semi detached house: 4
Detached house : 5

In [ ]:
#DAta Overview
data.head(10)

In [ ]:
data.columns

In [ ]:
# Detect null observation number and data type
data.info()

In [ ]:
#Overview for Numeric Variables 
# The data between 2008 to 2021 
data.describe().T

In [ ]:
# Word Compare 
#Ireland is among the top 5 countries in terms of per capita income.
#That is why I chose to make the comparison between the capitals of these countries. 

In [ ]:
## World Renting Visulation 
# it is to understand  both difference and similarity between  Europe Capitals Home Rents and  Dublin  

In [ ]:
#Alpha code to print europe map
country_codes = pd.read_csv("country_codes.tsv", sep='\t')
country_codes.head()

In [ ]:
#to check word data
world_data.head()

In [ ]:
#drop unnecessery columns
world_data.drop("TIME", axis=1,inplace=True)
world_data

In [ ]:
#relocation of rows and columns to make them suitable for testing
world_data_anova=world_data.transpose()
world_data_anova = world_data_anova.rename(columns=world_data_anova.iloc[0]).drop(world_data_anova.index[0])
world_data_anova.head()

In [ ]:
#Top 5 contry ( GDP per capita)
filtered_df = world_data_anova[['Denmark','Ireland','Norway', 'Switzerland','Luxembourg']]
filtered_df.head()

In [ ]:
#Normalization TEst 
#I applied Shapiro Wilk because I have just 20 rows. This test is more reliable on the data that is small
for column in filtered_df.columns:
    stat, p_value = shapiro(filtered_df[column])
    print(f"City {column}: Sattictic={stat:.4f}, p değeri={p_value:.4f}")

In [ ]:
#Denmark and Luxembourg have normal distribution
#IRelan Norway and Switzerland doesnt have normal distrubution

In [ ]:
#I used a one-way ANOVA because I compared 5 different countries and the values changed over time.
# ANOVA testi for series that have normal distrubation 
anova_result = f_oneway(world_data_anova['Ireland'], world_data_anova['Luxembourg'], 
                        world_data_anova['Denmark'], world_data_anova['Switzerland'], 
                        world_data_anova['Norway'])

print("ANOVA Test Result:")
print("F statisctic:", anova_result.statistic)
print("p value :", anova_result.pvalue)

In [ ]:
# Conclusion: p value < 0.05  H0 can not acceptable , at least one series is different from the others

In [ ]:
from scipy.stats import kruskal

#I used this test because it did not satisfy the normality assumption of some columns. 
# Kruskal-Wallis testi
test_stat, p_value = kruskal(*filtered_df.values.T)

# Sonuçları yazdırma
print("Kruskal-Wallis Test Result :")
print("Test statistic:", test_stat)
print("p value:", p_value)

In [ ]:
filtered_df = filtered_df.astype(float)

In [ ]:
#Mann Whitney U
from scipy.stats import mannwhitneyu
columns = ['Denmark','Ireland','Norway', 'Switzerland','Luxembourg']
for i, col in enumerate(columns):
    stat, p_value = mannwhitneyu(filtered_df['Ireland'], filtered_df[sütun])
    print(f" {col}: U statistic={stat:.4f}, p value={p_value:.4f}")

### As a result, there is no statistically significant difference between Irelan and Denmark, Ireland, Norway and Switzerland columns. However, it can be said that there is a statistically significant difference between the Luxembourg column and Ireland. These interpretations are based on p-values at the 0.05 significance level.
### I deliberately put the data for Ireland in because I wanted to check if the test result would come  as 1. so that I could be sure that the test was reliable.

In [ ]:
#Getting the data into the correct format for visualization
melted_data = pd.melt(world_data, id_vars='Country', var_name='Year', value_name='Rentt')
melted_data

In [ ]:
#Merge with alpha code
merged_data = melted_data.merge(country_codes, left_on='Country', right_on='Country', how='left')
merged_data

In [ ]:
#when we compare the European Capital's Rent Price , Ireland one od the most expensive country in the Europe.
# That's why I investigate  Dublin Rent Prices in this assignment

fig = px.choropleth(merged_data, 
                    locations="Alpha-3 code",
                    color="Rentt", 
                    hover_name="Country", 
                    # column to add to hover information
                    animation_frame="Year", 
                    # column on which to animate
                    color_continuous_scale=px.colors.sequential.Plasma )
                    
fig.update_layout(
    # add a title text for the plot
    title_text = 'REnt of Europe Capital Cities',
    # set projection style for the plot
    geo_scope = 'europe' 

)
fig.show()

## Data Cleaning 

In [ ]:
# Drop rows which doesnt have rent price information
data.dropna(subset=["VALUE"],inplace=True)

In [ ]:
#Drop first and fifth row
data=data.iloc[:,[1,2,3,4,6]]

In [ ]:
#To check the code
data.head()

In [ ]:
#MAp  Visualatisiton
# data for country visulation 
country_data = data[(data['Number of Bedrooms'] == 'All bedrooms')& (data["Property Type"]=="All property types")
                    &data["Location"].isin(["Dublin","Cork","Limerick","Galway","Waterford","Drogheda, Louth","Dundalk, Louth"])]
# & (df['sütun3'].isin(['Ccc', 'Ddd']))
# & (data['Property Type'] == 'All propert types')

In [ ]:
locat = sorted(country_data['Location'].unique())
locat

In [ ]:
import folium

# Dünya haritası oluşturun
world_map = folium.Map()

In [ ]:
sehirler = {'Cork': (51.8985, -8.4756),
            'Drogheda, Louth': (53.7189, -6.3561),
            'Dublin': (53.3498, -6.2603),
            'Dundalk, Louth': (54.0037, -6.4029),
            'Galway': (53.2707, -9.0568),
            'Limerick': (52.6638, -8.6267),
            'Waterford': (52.2593, -7.1101)}

# Şehir markörlerini haritaya ekle
for sehir, koordinatlar in sehirler.items():
    folium.Marker(location=koordinatlar, tooltip=sehir).add_to(world_map)

# Haritayı görüntüle
world_map


In [ ]:
import geopandas as gp
ulke = gp.read_file("countries.geojson")

In [ ]:
import pandas as pd
ulke.head(10) #ilk 10 satır 

In [ ]:
ireland = ulke[ulke['ADMIN'] == 'Ireland']
ireland

In [ ]:
#Cleaning unneccesary rows because We analize the houses by feature 
data=data.loc[ (data["Number of Bedrooms"].isin(["One bed","Two bed","Three bed","Four plus bed"]))
              & (data["Property Type"] != "All property types")]



In [ ]:
#To get information just about Dublin
dub_data=data[data["Location"].str.contains("Dublin")]

In [ ]:
locations = sorted(dub_data['Location'].unique())
locations

In [ ]:
# this analays based on Dublin areas ( Dublin 22 has any unique information so Dublin 22 Clondalkin is used for Dublin 22 area. 
#Dublin 22 consist of Clondalkin, Liffey Valley, Newcastle and Neilstown)

In [ ]:
dub_data=dub_data.loc[dub_data["Location"].isin(["Dublin 1","Dublin 2","Dublin 3","Dublin 4",
                           "Dublin 5","Dublin 6","Dublin 6W","Dublin 7","Dublin 8",
                           "Dublin 9","Dublin 10","Dublin 11","Dublin 12",
                           "Dublin 13","Dublin 14","Dublin 15","Dublin 16"
                           ,"Dublin 17","Dublin 18","Dublin 20","Dublin 22","Dublin 24","Dun Laoghaire, Dublin"
                          ,"Blackrock, Dublin","Swords, Dublin","Malahide, Dublin","Balbriggan, Dublin","Clondalkin, Dublin 22"])]

In [ ]:
dub_data=dub_data.reset_index(drop=True)

In [ ]:
# To learn any null values and 3324 rows
dub_data.info()

In [ ]:
#To know how many unique values has each collumns
for collumnss in ["Year","Number of Bedrooms","Property Type"]:
    print(collumnss)
    print(dub_data[collumnss].value_counts())

In [ ]:
#To detect any duplicated rows
duplicate=dub_data.duplicated()
print("Count of Dublicated rows :" ,duplicate.sum())


In [ ]:
#to detecet outliers

In [ ]:
sns.boxplot(x=dub_data["VALUE"])



In [ ]:
dub_data.boxplot(column="VALUE",by="Year")

In [ ]:
#Outliers
# 1st quartile
q1 = np.percentile(dub_data["VALUE"],25)
# 3rd quartile
q3 = np.percentile(dub_data["VALUE"],75)
  # IQR
IQR = q3 - q1
# Outlier step
outlier_step = IQR * 1.5
# detect outlier and
    # store 
outliers = dub_data[(dub_data["VALUE"] < q1 - outlier_step) | (dub_data["VALUE"] > q3 + outlier_step)]
print(outliers.head(20))

In [ ]:
#Distrubution of outliers
outliers["Location"].value_counts().head(50)

In [ ]:
for n in var :
    plot_hist(n)

In [ ]:
#127 rows
outliers.info()

In [ ]:
#outliar percantege ( outliers/full data) %3 
print(127/3324)

In [ ]:
#percentage of outliers by Number of Bedrooms
x=dub_data["Number of Bedrooms"].value_counts()
y=outliers["Number of Bedrooms"].value_counts()
percentage=y/x
percentage.sort_values(ascending=False)

In [ ]:
#percentage of outliers by location
x=dub_data["Location"].value_counts()
y=outliers["Location"].value_counts()
percentage=y/x
percentage.sort_values(ascending=False)

In [ ]:
##percentage of outliers by Property Type
x=dub_data["Property Type"].value_counts()
y=outliers["Property Type"].value_counts()
percentage=y/x
percentage.sort_values(ascending=False)

In [ ]:
##percentage of outliers by Year
x=dub_data["Year"].value_counts()
y=outliers["Year"].value_counts()
percentage=y/x
percentage.sort_values(ascending=False)

### The study will continue with outlier values(%3). As a result of the analysis, it is seen that the outlier values are realized in the location, kind of house and number of rooms, which are expected to have higher values. Also, outlier values have intensified in recent years. It is an expected situation due to inflation. Now The outlier values will not be intervened to avoid information loss. 
### distribution statistics and visualition tools will be checked to make a final decision


##  Variable Analysis

In [ ]:
# String variables is changed with numeric variables . The numerical values will be assigned according to the average value.

dub_data.groupby("Number of Bedrooms", as_index=False)['VALUE'].mean().sort_values(by="VALUE",ascending=False)

In [ ]:
dub_data["Number of Bedrooms"] = dub_data["Number of Bedrooms"].replace({'One bed': 1,
                                                                       "Two bed": 2,
                                                                       "Three bed" : 3,
                                                                       "Four plus bed": 4 })

In [ ]:
dub_data.groupby("Property Type", as_index=False)['VALUE'].mean().sort_values(by="VALUE",ascending=False)

In [ ]:
dub_data["Property Type"] = dub_data["Property Type"].replace({'Other flats': 1,
                                                                       "Apartment": 2,
                                                                       "Terrace house" : 3,
                                                                       "Semi detached house": 4,
                                                                        "Detached house" : 5})

In [ ]:
dub_data.head(10)

In [ ]:
# First look for data. 
dub_data.describe().T

## Visualisation

In [ ]:
# this visualisation method used for to see different between the categories also, frequency, min and max value and median
# the houses that has 4 or more bedrooms , rent price has big variance . it may have many outliers 
# other count of bedrooms mod and median values look like close
sns.violinplot(data=dub_data, x="Number of Bedrooms", y="VALUE")
plt.title("Violin Plot of Number of Bedrooms")

In [ ]:
# Apartments  may has some outliers
# number 3 or 4 looks pretty close each other as mean,mod and frequency
sns.violinplot(data=dub_data, x="Property Type", y="VALUE")
plt.title("Violin Plot of Property Type")

In [ ]:
# as both the number of property types and the number of bedrooms increase in value, the value increases
sns.scatterplot(
    data=dub_data, x="Number of Bedrooms", y="Property Type", hue="VALUE", size="VALUE",
    sizes=(20, 200)
)
plt.title("Scatter Plot of Number of Bedrooms and Property Type")

In [ ]:
#Number 4 has biggest variance also average rent different each groups
g = sns.catplot(
    data=dub_data, x="Number of Bedrooms", y="VALUE", 
    kind="bar", height=4, aspect=.9,
)
g.set_axis_labels("", "Rent")
g.despine(left=True)
plt.title("Bar Plot of Number of Bedrooms")

In [ ]:
#Number 5 has biggest variance also average rent different each groups
g = sns.catplot(
    data=dub_data, x="Property Type", y="VALUE", 
    kind="bar", height=4, aspect=.9,
)
g.set_axis_labels("", "Rent")
g.despine(left=True)
plt.title("Violin Plot of Property Type")

In [ ]:
#propert type 4 and 3 have similar properties. therefore they will be combined into a single group
dub_data.groupby("Property Type", as_index=False)['VALUE'].agg(["mean","var","median"]).sort_values(by="mean",ascending=False)

In [ ]:
# corelattion before grouping 
dub_data["VALUE"].corr(dub_data["Property Type"])

In [ ]:
dub_data["Property Type"]= [3 if i==3 else 3 if i==4 else i for i in dub_data["Property Type"]]

In [ ]:
g = sns.catplot(
    data=dub_data, x="Property Type", y="VALUE", 
    kind="bar", height=4, aspect=.9,
)
g.set_axis_labels("", "Rent")
g.despine(left=True)
plt.title("Violin Plot of New Property Type")

In [ ]:
# as number of years increase, the Value  increases (except 2009-2010 \ 2021)
#rents have started to exceed 3000 euros since 2015
sns.scatterplot(data=dub_data, x="Year", y="VALUE")
plt.title("SCatter Plot of VALUE by Year")

In [ ]:
sns.scatterplot(data=dub_data, x="Year", y="VALUE", hue="Number of Bedrooms")
plt.title("Scatter Plot of VALUE by Year with Number of Beedroms ")

In [ ]:
# heat map for correlation
#number of bedrooms has the highest corellation with VALUE
#property type has the lowest corellation with VALUE
#after grouping , the corellation increased from 36 to 40
sns.heatmap(dub_data.corr(),annot=True,fmt=".3f")
plt.title("Corellation Matrix of Data")
plt.show()

### Distrubation Analys

In [ ]:
sns.distplot(dub_data["VALUE"])
plt.title("Distrubation plot of VALUE ")

In [ ]:
skewness = skew(dub_data['VALUE'])
kurtosis = kurtosis(dub_data['VALUE'])

print("skewness statistic: ", skewness)
print("kurtosis statistic: ", kurtosis)

In [ ]:
# kurtosis close 3 . the data has normal distribation (mesokurtic)
# skewness 1.4 . the data has positively skewed distribation

In [ ]:
statis, pvalue = shapiro(dub_data["VALUE"])
print('Test statistic: ', statis)
print('p-value: ', pvalue)

In [ ]:
#This method is used for small data 
# p-value < 0.05 data set does not follow a normal distribution

In [ ]:
jb_statis, pvalue = jarque_bera(dub_data["VALUE"])
print(pvalue)

In [ ]:
#p-value < 0.05 data set does not follow a normal distribution

In [ ]:
# Normalization with Min Max Scaler
scaler = MinMaxScaler()
MM_norm_data= scaler.fit_transform(dub_data[["VALUE"]])
MM_norm_data=pd.DataFrame(MM_norm_data,columns=["VALUE"])

In [ ]:
print(type(MM_norm_data))

In [ ]:
sns.distplot(dub_data["VALUE"])
plt.title("Distrubation plot of VALUE After Min Max Scaler ")

In [ ]:
skewness1 = skew(MM_norm_data['VALUE'])
#kurto_val = kurtosis(MM_norm_data['VALUE'])
print("skewness statistic: ", skewness1)
#print("kurtosis statistic: ", kurto_val)

In [ ]:
# skewness 1.4 . the data has positively skewed distribation

In [ ]:
jb_statis, pvalue = jarque_bera(dub_data["VALUE"])
print(pvalue)

In [ ]:
# Robust transform is more effective on data with outliers
scaler = RobustScaler()
RBS_data= scaler.fit_transform(dub_data[["VALUE"]])
RBS_data=pd.DataFrame(RBS_data,columns=["VALUE"])

In [ ]:
sns.distplot(RBS_data["VALUE"])
plt.title("Distrubation plot of VALUE after Robust Transform ")

In [ ]:
skew2 = skew(RBS_data['VALUE'])
#kurto2 = kurtosis(RBS_data["VALUE"])
print("skewness statistic: ", skew2)
#print("kurtosis statistic: ", kurto2)

In [ ]:
jb_statis, pvalue = jarque_bera(RBS_data["VALUE"])
print(pvalue)

In [ ]:
#p-value < 0.05 data set does not follow a normal distribution

In [ ]:
# Removing Outliers

In [ ]:
dubnew_data = dub_data[(dub_data["VALUE"] >= q1 - outlier_step) & (dub_data["VALUE"] <= q3 + outlier_step)]

In [ ]:
# 127 datas was removed
dubnew_data.describe().T

In [ ]:
sns.distplot(dubnew_data["VALUE"])
plt.title("Distrubation plot of VALUE without outliers ")

In [ ]:
dubnew_data.info()

In [ ]:
skewness_new = skew(dubnew_data['VALUE'])
#kurtosis_new = kurtosis(dubnew_data['VALUE'])

print("skewness statistic: ", skewness_new)
#print("kurtosis statistic: ", kurtosis_new)

In [ ]:
# skewness 0.6 . the data has symmetrical distribation

In [ ]:
#create dummy variable
dummy_var=pd.get_dummies(dubnew_data["Location"])
#concat dummy variable
dubdummy_data=pd.concat([dubnew_data,dummy_var],axis=1)


In [ ]:
dubdummy_data.head()

In [ ]:
# Normalization with Min Max Scaler
scaler = MinMaxScaler()
dubdummy_data["Year"]= scaler.fit_transform(dubdummy_data[["Year"]])


In [ ]:
#drop Location column
dubdummy_data.drop("Location",axis=1,inplace=True)

In [ ]:
# TRain TEst Split 
X_train = dubdummy_data.drop(labels = "VALUE",axis=1)
y_train = dubdummy_data["VALUE"]
#test size %20 test data %80 percent train data
#random state controls the shuffling process (number 44 is my hometown number in Turkey)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size =0.20, random_state = 44 )
print("X_train",len(X_train))
print("X_test",len(X_test))
print("y_train",len(y_train))
print("y_test",len(y_test))


## Modelling

In [ ]:
# Lineer REgression , Decission Tree, Random Forest Regressor are using for anlaysis

### Lineer Regression

In [ ]:
#x=x_train.iloc[:,1:8].values
#y=y_train.VALUE.values.reshape(-1,1)
print(X_train.shape)
print(y_train.shape)
linreg = LinearRegression()
linreg.fit(X_train,y_train)
b0_=linreg.intercept_
b1_=linreg.coef_
print("b0_ :",b0_)
print("b1_ ;", b1_)
print("R_2 score :", linreg.score(X_train,y_train))

In [ ]:
import statsmodels.api as sm

#X_train_cons=sm.add_constant(X_train)

# Note the difference in argument order
model = sm.OLS(y_train, X_train).fit()
predictions = model.predict(X_train) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
# The variable "Clondalkin, Dublin 22 " is statistically insignificant P>|t| > 0.05 . So I dropClondalkin, Dublin 22 column from the dataset
x = X_train.drop(["Clondalkin, Dublin 22"], axis=1)

# Note the difference in argument order
model = sm.OLS(y_train, x).fit()
predictions = model.predict(x) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
# The variable "Dublin 24 " is statistically insignificant P>|t| > 0.05 . So I drop Dublin 24 column from the dataset
x = X_train.drop(["Clondalkin, Dublin 22","Dublin 24"], axis=1)

# Note the difference in argument order
model = sm.OLS(y_train, x).fit()
predictions = model.predict(x) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
# The variable "Dublin 15 " is statistically insignificant P>|t| > 0.05 . So I drop Dublin 15 column from the dataset
x = X_train.drop(["Clondalkin, Dublin 22","Dublin 24","Dublin 15"], axis=1)

# Note the difference in argument order
model = sm.OLS(y_train, x).fit()
predictions = model.predict(x) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
# The variable "Dublin 10 " is statistically insignificant P>|t| > 0.05 . So I drop Dublin 10 column from the dataset
x = X_train.drop(["Clondalkin, Dublin 22","Dublin 24","Dublin 15","Dublin 10"], axis=1)

# Note the difference in argument order
model = sm.OLS(y_train, x).fit()
predictions = model.predict(x) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
# The variable "Dublin 17 " is statistically insignificant P>|t| > 0.05 . So I drop Dublin 17 column from the dataset
x = X_train.drop(["Clondalkin, Dublin 22","Dublin 24","Dublin 15","Dublin 10","Dublin 17"], axis=1)

# Note the difference in argument order
model = sm.OLS(y_train, x).fit()
predictions = model.predict(x) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
# The variable "Swords, Dublin " is statistically insignificant P>|t| > 0.05 . So I drop Swords, Dublin column from the dataset
x = X_train.drop(["Clondalkin, Dublin 22","Dublin 24","Dublin 15","Dublin 10","Dublin 17","Swords, Dublin"], axis=1)

# Note the difference in argument order
model = sm.OLS(y_train, x).fit()
predictions = model.predict(x) # make the predictions by the model

# Print out the statistics
model.summary()

In [ ]:
# All variables is statistically significant.
#Also f prob < 0.05 the last model and dependent variable is statistically significant.

In [ ]:
X_test_lineer = X_test.drop(["Clondalkin, Dublin 22","Dublin 24","Dublin 15","Dublin 10","Dublin 17","Swords, Dublin"], axis=1)
prediction_test=model.predict(X_test_lineer)
mse = mean_squared_error(y_test, prediction_test)
mae = mean_absolute_error(y_test, prediction_test)
R_2 = r2_score(y_test, prediction_test)
print("mse:" ,mse)
print("mae:", mae)
print("R_2:", R_2)

In [ ]:
#overfitting problem
#mse: 44165.21199799048
#mae: 170.72865199944485
#R_2: 0.7554666834196191